<a href="https://colab.research.google.com/github/CaioCesar999/DNC_Port/blob/main/Din%C3%A2mica_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SETUP**

## Spark UI

In [1]:
!pip install -q pyspark
!pip install -q pyngrok

     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 51.5 MB/s 
     |████████████████████████████████| 745 kB 9.8 MB/s 


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.ui.port', '4050').getOrCreate()
spark

In [3]:
# Fazer login no site https://dashboard.ngrok.com/get-started/setup para obter autenticação própria
ngrok_token = '27WqUGZ1SkzPl4bQunMXJyasbWZ_5pCgLCsppL7ufk9rzK7j4'

In [4]:
get_ipython().system_raw(f'ngrok authtoken {ngrok_token}')
get_ipython().system_raw('ngrok http 4050 &')
!sleep 3
print('URL para interface Spark:')
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

URL para interface Spark:


## Libraries

In [5]:
import pandas as pd
from google.colab import files

import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Load data

In [6]:
# Load file
_ = files.upload()

Saving vgsales.csv to vgsales.csv


## Data Description

This dataset contains records of popular video games in North America, Japan, Europe and other parts of the world. Every video game in this dataset has at least 100k global sales.

[Source](https://www.kaggle.com/datasets/gregorut/videogamesales/code?datasetId=284&sortBy=voteCount) of dataset.

## Data Dictionary

| Column       | Explanation                                               |
|:-------------|:----------------------------------------------------------|
| Rank         | Ranking of overall sales                                  |
| Name         | Name of the game                                          |
| Platform     | Platform of the games release (i.e. PC,PS4, etc.)         |
| Year         | Year the game was released in                             |
| Genre        | Genre of the game                                         |
| Publisher    | Publisher of the game                                     |
| NA_Sales     | Number of sales in North America (in millions)            |
| EU_Sales     | Number of sales in Europe (in millions)                   |
| JP_Sales     | Number of sales in Japan (in millions)                    |
| Other_Sales  | Number of sales in other parts of the world (in millions) |
| Global_Sales | Number of total sales (in millions)                       |

# Questões

## Questão 1

Leia os dados com Spark, certificando-se que

1. A tabela tenha duas partições e

2. As colunas da tabela tenham este respectivo schema:

|Column|Data type|
|---|---|
|Rank|integer|
|Name|string|
|Platform|string|
|Year|integer|
|Genre|string|
|Publisher|string|
|NA_Sales|double|
|EU_Sales|double|
|JP_Sales|double|
|Other_Sales|double|
|Global_Sales|double|

In [16]:
df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



In [8]:
df = spark.read.csv('vgsales.csv', header=True, inferSchema=True) 

In [9]:
df.head()

Row(Rank=1, Name='Wii Sports', Platform='Wii', Year='2006', Genre='Sports', Publisher='Nintendo', NA_Sales=41.49, EU_Sales=29.02, JP_Sales=3.77, Other_Sales=8.46, Global_Sales=82.74)

In [14]:
df = df.repartition(2) 

In [17]:
df = df.withColumn('Year', F.col('Year').cast('integer')) 

In [18]:
df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



## Questão 2

Construa uma tabela com:
- contagem das observações
- média
- desvio padrão
- valor mínimo
- valor máximo

para todas as coluna de vendas (que possuem "Sales" no nome).

**Dica:** uma **única função** consegue calcular todos estes valores.

In [21]:
df.describe(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']).show()

+-------+------------------+-------------------+-------------------+--------------------+------------------+
|summary|          NA_Sales|           EU_Sales|           JP_Sales|         Other_Sales|      Global_Sales|
+-------+------------------+-------------------+-------------------+--------------------+------------------+
|  count|             16598|              16598|              16598|               16598|             16598|
|   mean|0.2646674298108155|0.14665200626581024|0.07778166044101428|0.048063019640918934|0.5374406555006714|
| stddev|0.8166830292988791| 0.5053512312869121| 0.3092906480822032| 0.18858840291271461|1.5550279355699124|
|    min|               0.0|                0.0|                0.0|                 0.0|              0.01|
|    max|             41.49|              29.02|              10.22|               10.57|             82.74|
+-------+------------------+-------------------+-------------------+--------------------+------------------+



## Questão 3

Para cada variável categórica do tipo string, calcule quantas categorias distintas estão presentes.

In [26]:
if df.dtypes == 'string':
  for i in df.columns:
    df.select('Category').distinct().show(truncate=False)
df.head()  

Row(Rank=3296, Name='Dora the Explorer: Super Spies', Platform='GBA', Year=2004, Genre='Platform', Publisher='Gotham Games', NA_Sales=0.44, EU_Sales=0.16, JP_Sales=0.0, Other_Sales=0.01, Global_Sales=0.61)

In [28]:
#columnList = [item[0] for item in df if item[1].startswith('string')] 
for tipo in df.dtypes:
  if tipo[1] == 'string':
    df.select(F.countDistinct(tipo[0])).distinct().show(truncate=False) 

+--------------------+
|count(DISTINCT Name)|
+--------------------+
|11493               |
+--------------------+

+------------------------+
|count(DISTINCT Platform)|
+------------------------+
|31                      |
+------------------------+

+---------------------+
|count(DISTINCT Genre)|
+---------------------+
|12                   |
+---------------------+

+-------------------------+
|count(DISTINCT Publisher)|
+-------------------------+
|579                      |
+-------------------------+



In [31]:
df.select(*[F.count_distinct(col) for col in ['Name','Platform','Genre','Publisher']]).show() 

+--------------------+------------------------+---------------------+-------------------------+
|count(DISTINCT Name)|count(DISTINCT Platform)|count(DISTINCT Genre)|count(DISTINCT Publisher)|
+--------------------+------------------------+---------------------+-------------------------+
|               11493|                      31|                   12|                      579|
+--------------------+------------------------+---------------------+-------------------------+



## Questão 4

Qual plataforma vendeu mais, mundialmente, considerando todo o período histórico diponível dos dados?  
Mostre as top 10 plataformas em vendas.

In [34]:
(
  df
 .groupby('Platform')
 .agg(
    F.sum('Global_Sales').alias('Global_Sales_sum'),
 )
 .orderBy(F.desc('Global_Sales_sum'))
 .show(10, truncate=False)
) 

+--------+------------------+
|Platform|Global_Sales_sum  |
+--------+------------------+
|PS2     |1255.6399999999956|
|X360    |979.9600000000003 |
|PS3     |957.8400000000025 |
|Wii     |926.7099999999973 |
|DS      |822.4899999999982 |
|PS      |730.6599999999992 |
|GBA     |318.49999999999943|
|PSP     |296.2799999999996 |
|PS4     |278.0999999999999 |
|PC      |258.82000000000085|
+--------+------------------+
only showing top 10 rows



## Questão 5

Faça uma tabela com os jogos que aparecem em múltiplas plataformas, ordene de forma que os jogos com mais plataformas apareçam primeiro e responda:
- Qual jogo aparece em mais plataformas? Em quantas plataformas?

In [38]:
(
  df
 .groupby('Name')
 .agg(
    F.count_distinct('Platform').alias('Platform_count_distinct')
 )
 .where('Platform_count_distinct > 1')
 .orderBy(F.desc('Platform_count_distinct'))
 .show(10, truncate=False)
) 


+----------------------------+-----------------------+
|Name                        |Platform_count_distinct|
+----------------------------+-----------------------+
|Need for Speed: Most Wanted |10                     |
|Madden NFL 07               |9                      |
|LEGO Marvel Super Heroes    |9                      |
|Ratatouille                 |9                      |
|FIFA 14                     |9                      |
|Terraria                    |8                      |
|Madden NFL 08               |8                      |
|Monopoly                    |8                      |
|Lego Batman 3: Beyond Gotham|8                      |
|Angry Birds Star Wars       |8                      |
+----------------------------+-----------------------+
only showing top 10 rows



**Resposta:** Need for Speed: Most Wanted. Aparece em 10 plataformas.

## Questão 6

Utilize a API do Pandas no Spark para calcular a soma das vendas globais para cada ano e gênro de jogo. Faça então um gráfico de linhas com os anos no eixo `x`, as vendas no eixo `y`, de forma que cada linha corresponda a um gênero de jogo.

**Dica:** após o cálculo, passar os dados para Pandas antes da plotagem, ou plotar diretamente aproveitando os métodos da classe `pyspark.pandas.DataFrame`.

Analise o gráfico e responda:
- Entre 1980 e 1990, quais gêneros mais venderam?
- Entre 2000 e 2015, quais gêneros mais venderam?

In [40]:
(
  df
 .groupby('Year','Genre')
 .agg(
    F.sum('Global_Sales').alias('Global_Sales_sum')
 )
 .orderBy(F.desc('Global_Sales_sum'))
 .show(truncate=False)
) 


+----+-------+------------------+
|Year|Genre  |Global_Sales_sum  |
+----+-------+------------------+
|2009|Action |139.36000000000007|
|2009|Sports |138.51999999999992|
|2008|Action |136.39000000000004|
|2006|Sports |136.16000000000003|
|2013|Action |125.22            |
|2012|Action |122.04            |
|2011|Action |118.95999999999995|
|2010|Action |117.63999999999999|
|2007|Action |106.5             |
|2011|Shooter|99.35999999999999 |
|2014|Action |99.02000000000001 |
|2007|Sports |98.20000000000005 |
|2010|Misc   |96.85999999999999 |
|2008|Sports |95.34000000000003 |
|2010|Sports |92.53000000000002 |
|2007|Misc   |92.27000000000001 |
|2008|Misc   |87.02999999999999 |
|2002|Action |86.77000000000002 |
|2005|Action |85.69000000000003 |
|2010|Shooter|77.41             |
+----+-------+------------------+
only showing top 20 rows



In [50]:
df.pandas_api().groupby(['Year','Genre'])[['Global_Sales']].sum().sort_index().reset_index().plot.line(x='Year', y='Global_Sales', color = 'Genre') 

**Resposta:**
- Entre 1980 e 1990 sobressaíram as vendas de plataforma, tiro (shooter) e puzzle.
- Entre 2000 e 2015 teve vendas elevadas de jogos de ação e esporte.

## Questão 7

Registre a tabela usando `createOrReplaceTempView` e faça uso da linguagem SQL criar uma tabela que:
- Considere apenas os anos da década de 90 e 
- Agrupe por ano para responder quantos % cada região teve do total de vendas (vendas globais).

Salve o resultado desta query em uma variável chamada `df_questao7`.

Após isso, execute o seguinte comando `df_questao7.pandas_api().set_index('YEAR').style.background_gradient(cmap='Oranges')` e responda:
- Qual/quais regiões tiveram, relativamente (comparado às vendas globais), mais vendas no fim da década de noventa do que no início?








In [68]:
df.createOrReplaceTempView('Table')
Query = """Select Year,
                sum(Global_Sales) as GS,
                round(sum(NA_Sales)/sum(Global_Sales)*100,2) as pct_NA, 
                round(sum(EU_Sales)/sum(Global_Sales)*100,2) as pct_EU, 
                round(sum(JP_Sales)/sum(Global_Sales)*100,2) as pct_JP, 
                round(sum(Other_Sales)/sum(Global_Sales)*100,2) as pct_Other
                From Table 
           Where Year Between 1990 and 2000
           Group by Year
           Order by Year DESC;        
"""
df_questao7 = spark.sql(Query)
df_questao7.show() 

+----+------------------+------+------+------+---------+
|Year|                GS|pct_NA|pct_EU|pct_JP|pct_Other|
+----+------------------+------+------+------+---------+
|2000|            201.56| 46.88| 26.17| 21.22|     5.77|
|1999|251.27000000000004| 50.17| 24.94| 20.83|      4.0|
|1998|256.47000000000014| 50.05| 26.08| 19.51|      4.3|
|1997|            200.98| 47.14| 24.04| 24.32|     4.54|
|1996|199.15000000000003| 43.57| 23.73| 28.84|     3.86|
|1995|             88.11| 28.17| 16.91| 51.92|      3.0|
|1994| 79.16999999999996| 35.56| 18.79| 42.93|     2.78|
|1993|             45.98| 32.88| 10.11| 55.09|     1.94|
|1992|             76.16| 44.47| 15.38| 37.96|     2.17|
|1991|32.230000000000004| 39.59| 12.26| 45.86|      2.3|
|1990|             49.39| 51.55| 15.45| 30.13|     2.83|
+----+------------------+------+------+------+---------+



In [69]:
df_questao7.pandas_api().set_index('Year').style.background_gradient(cmap='Oranges')

,GS,pct_NA,pct_EU,pct_JP,pct_Other
Year,,,,,
2000,201.560000,46.880000,26.170000,21.220000,5.770000
1999,251.270000,50.170000,24.940000,20.830000,4.000000
1998,256.470000,50.050000,26.080000,19.510000,4.300000
1997,200.980000,47.140000,24.040000,24.320000,4.540000
1996,199.150000,43.570000,23.730000,28.840000,3.860000
1995,88.110000,28.170000,16.910000,51.920000,3.000000
1994,79.170000,35.560000,18.790000,42.930000,2.780000
1993,45.980000,32.880000,10.110000,55.090000,1.940000
1992,76.160000,44.470000,15.380000,37.960000,2.170000


**Resposta:** A região da Europa e "Outras regiões" tiverem um aumento notável quando comparado ao ínicio da década de 90.

## Questão 8

Calcule quantas vezes cada região teve vendas superiores às demais.

In [72]:
df.createOrReplaceTempView('Table')
Query = """Select Year,
                sum(Global_Sales) as GS,
                round(sum(NA_Sales)/sum(Global_Sales)*100,2) as pct_NA, 
                round(sum(EU_Sales)/sum(Global_Sales)*100,2) as pct_EU, 
                round(sum(JP_Sales)/sum(Global_Sales)*100,2) as pct_JP, 
                round(sum(Other_Sales)/sum(Global_Sales)*100,2) as pct_Other
                From Table 
                Group by Year
                Order by Year DESC;        
"""
df_questao8 = spark.sql(Query)
df_questao8.show() 

+----+------------------+------+------+------+---------+
|Year|                GS|pct_NA|pct_EU|pct_JP|pct_Other|
+----+------------------+------+------+------+---------+
|2020|              0.29|  93.1|   0.0|   0.0|      6.9|
|2017|              0.05|   0.0|   0.0| 100.0|      0.0|
|2016| 70.92999999999998| 31.95| 37.73| 19.31|    10.93|
|2015| 264.4400000000001| 38.88| 36.95| 12.75|    11.35|
|2014| 337.0500000000003| 39.15| 37.28| 11.71|    11.87|
|2013| 368.1100000000001| 42.04| 34.17| 12.93|    10.82|
|2012|363.53999999999985| 42.63| 32.67| 14.23|     10.4|
|2011| 515.9900000000014| 46.72| 32.45| 10.28|    10.54|
|2010|            600.45| 50.67| 29.43|  9.91|     9.98|
|2009| 667.2999999999994| 50.78| 28.71|  9.27|     11.2|
|2008| 678.8999999999977| 51.77| 27.16|  8.88|    12.14|
|2007| 611.1299999999991| 51.06| 26.26|  9.87|     12.7|
|2006| 521.0399999999998|  50.5|  24.8| 14.15|    10.45|
|2005| 459.9400000000002| 52.75| 26.51|  11.8|     8.82|
|2004| 419.3100000000001| 53.08

In [86]:
df1 = df.withColumn('max_sale', F.greatest(*['NA_Sales','EU_Sales','JP_Sales','Other_Sales'])).withColumn(
     'Regiao_Maior_Venda',
     F.when(F.col('NA_Sales') == F.col('max_sale'), 'NA_Sales')
     .when(F.col('EU_Sales') == F.col('max_sale'), 'EU_Sales')
     .when(F.col('JP_Sales') == F.col('max_sale'), 'JP_Sales')
     .otherwise('Other_Sales')
     )

df1.show()


+-----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+--------+------------------+
| Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|max_sale|Regiao_Maior_Venda|
+-----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+--------+------------------+
| 3296|Dora the Explorer...|     GBA|2004|    Platform|        Gotham Games|    0.44|    0.16|     0.0|       0.01|        0.61|    0.44|          NA_Sales|
| 5695|Dynasty Warriors ...|     PS2|2004|      Action|          Tecmo Koei|     0.0|     0.0|    0.32|        0.0|        0.32|    0.32|          JP_Sales|
| 4986|San Francisco Rus...|      PS|1997|      Racing|      GT Interactive|    0.21|    0.15|     0.0|       0.03|        0.38|    0.21|          NA_Sales|
| 1281|WWF WrestleMania ...|     N64|1999|    Fighting|   

In [87]:
df1.groupby('Regiao_Maior_Venda').count().show()

+------------------+-----+
|Regiao_Maior_Venda|count|
+------------------+-----+
|          EU_Sales| 2380|
|          JP_Sales| 4029|
|          NA_Sales|10113|
|       Other_Sales|   76|
+------------------+-----+



## Questão 9

Construe uma tabela que mostre a diferença do total de vendas em um década com a década anterior e responda:
- Qual década apresentou a **menor diferença** comparada à década anterior?

In [104]:
#df.withColumn('Decade', F.floor(F.col('Year')/10)*10).show()
df2 = (
    df.where('Year is not null')
  .withColumn('Decade', F.floor(F.col('Year')/10)*10)
  .groupby('Decade')
  .sum('Global_Sales')
  .orderBy('Decade')
  )

df2.show()

+------+------------------+
|Decade| sum(Global_Sales)|
+------+------------------+
|  1980|376.58000000000004|
|  1990|1278.9099999999992|
|  2000|4644.0199999999695|
|  2010|2520.5599999999868|
|  2020|              0.29|
+------+------------------+



In [105]:
w = Window.orderBy('Decade')

(
  df
 .where('Year IS NOT NULL')
 .withColumn('Decade', F.floor(F.col('Year')/10)*10) # possível fazer com when, porém mais código necessário
 .groupby('Decade')
 .agg(F.mean('Global_Sales').alias('global_sales_mean'))
 .withColumn('global_sales_mean_lag', F.lag('global_sales_mean').over(w))
 .withColumn('Delta_global_sales_mean', F.col('global_sales_mean') - F.col('global_sales_mean_lag'))
 .orderBy('Decade')
 .show()
) 

+------+-------------------+---------------------+-----------------------+
|Decade|  global_sales_mean|global_sales_mean_lag|Delta_global_sales_mean|
+------+-------------------+---------------------+-----------------------+
|  1980| 1.8369756097560979|                 null|                   null|
|  1990|   0.72295647258338|   1.8369756097560979|     -1.114019137172718|
|  2000| 0.5043462206776683|     0.72295647258338|    -0.2186102519057117|
|  2010|0.48999999999999744|   0.5043462206776683|   -0.01434622067767...|
|  2020|               0.29|  0.48999999999999744|   -0.19999999999999746|
+------+-------------------+---------------------+-----------------------+



## Questão 10 - PLUS

Utilizando apenas a sintaxe do pyspark, faça uma função que calcule a moda do dataframe para um grupo de colunas que o usuário vai especificar. Além da moda, o dataframe de saída deve conter a quantidade de vezes que a moda aparece, e tambem se é multimodal.

Sua função deve receber como entrada:
- O pyspark dataframe
- A coluna-alvo que desejamos saber a moda e
- Os grupos (colunas do dataframe) que iremos considerar para calcular a moda 

O resultado será uma linha por grupo, contendo a identificação do grupo, a moda, a quantidade de vezes que apareceu, e se é um caso multimodal.

Por exemplo, considerando o seguinte dataframe:

```
df_test = spark.createDataFrame([
  (2.4,'A','A1'), (2.4,'A','A1'), (2.5,'A','A1'), (2.6,'A','A1'), (2.7,'A','A1'),
  (2.4,'B','A1'), (2.5,'B','A1'), (2.5,'B','A1'), (2.6,'B','A1'), (2.4,'B','A1')
], ('values','inner_group', 'main_group')).select('main_group', 'inner_group', 'values')

df_test.show()
```

```
+----------+-----------+------+
|main_group|inner_group|values|
+----------+-----------+------+
|        A1|          A|   2.4|
|        A1|          A|   2.4|
|        A1|          A|   2.5|
|        A1|          A|   2.6|
|        A1|          A|   2.7|
|        A1|          B|   2.4|
|        A1|          B|   2.5|
|        A1|          B|   2.5|
|        A1|          B|   2.6|
|        A1|          B|   2.4|
+----------+-----------+------+
```

Ao aplicar a função:  
`calculate_mode(df=df_test, target_col='values', group_cols=['main_group','inner_group']).show()`

o retorno deve ser:

```
+----------+-----------+----+-----------+----------+
|main_group|inner_group|mode|mode_counts|multimodal|
+----------+-----------+----+-----------+----------+
|        A1|          A| 2.4|          2|     false|
|        A1|          B| 2.4|          2|      true|
+----------+-----------+----+-----------+----------+
```

pois no grupo `A1-A` o valor 2.4 é o que mais aparece (2x), e nenhum outro valor aparece duas vezes também. Já no grupo `A1-B` o valor 2.4 também é o que mais aparece (2X), porém em conjunto com o 2.5, que também aparece duas 2X. Neste caso, devemos reportar apenas uma das modas e informar que é multimodal.

In [106]:
df_test = spark.createDataFrame([
  (2.4,'A','A1'), (2.4,'A','A1'), (2.5,'A','A1'), (2.6,'A','A1'), (2.7,'A','A1'),
  (2.4,'B','A1'), (2.5,'B','A1'), (2.5,'B','A1'), (2.6,'B','A1'), (2.4,'B','A1')
], ('values','inner_group', 'main_group')).select('main_group', 'inner_group', 'values')

df_test.show()

+----------+-----------+------+
|main_group|inner_group|values|
+----------+-----------+------+
|        A1|          A|   2.4|
|        A1|          A|   2.4|
|        A1|          A|   2.5|
|        A1|          A|   2.6|
|        A1|          A|   2.7|
|        A1|          B|   2.4|
|        A1|          B|   2.5|
|        A1|          B|   2.5|
|        A1|          B|   2.6|
|        A1|          B|   2.4|
+----------+-----------+------+



In [107]:
def calculate_mode(df, target_col, group_cols=None):
  """Calculate mode

  Parameters
  ----------
  df : pyspark.DataFrame
    The dataframe in which the function will be applied
  target_col : str
    Column name in which the mode will be calculated from
  group_cols : list
    List column names to make the group. If None, no grouping will be considered.

  Returns
  -------
  output: pyspark.DataFrame
    Dataframe with group identification (one per line), the mode, value counts,
    and a column specifying if the group has more than one mode.
  """

  if group_cols is None:
    window_vals = Window.partitionBy('count')
    df_mode = df.groupby(target_col).count()
    max_count = df_mode.select(F.max('count')).collect()[0][0]

    df_mode = df_mode.filter(F.col('count')==max_count)
    nrows = df_mode.count()

    df_mode = df_mode.select(F.col(target_col).alias('mode'), F.col('count').alias('mode_counts'), (F.lit(nrows)>1).alias('multimodal'))

  else:
    window_grp = Window.partitionBy(*group_cols)
    window_grp_vals = Window.partitionBy(*group_cols, target_col)

    df_mode = (
      df
      .withColumn('counts', F.count(F.col(target_col)).over(window_grp_vals))
      .withColumn('counts_max', F.max('counts').over(window_grp))
      .filter(f'counts = counts_max')
      .dropDuplicates(subset=[*group_cols, target_col])
      .withColumn('multimodal', F.count(target_col).over(window_grp)>1)
      .withColumnRenamed(target_col, 'mode')
      .withColumnRenamed('counts_max', 'mode_counts')
      .dropDuplicates(subset=[*group_cols])
      .select(*group_cols, 'mode', 'mode_counts', 'multimodal')
    )
  return df_mode

In [108]:
calculate_mode(df_test, 'values', ['main_group','inner_group']).show()

+----------+-----------+----+-----------+----------+
|main_group|inner_group|mode|mode_counts|multimodal|
+----------+-----------+----+-----------+----------+
|        A1|          A| 2.4|          2|     false|
|        A1|          B| 2.4|          2|      true|
+----------+-----------+----+-----------+----------+

